# Modified Multiplication Table Method

## Definir fonction

fonction objective

In [1]:
from email.utils import collapse_rfc2231_value

import sympy as sp
import numpy as np
import math
import copy

global variables
variables=[]
dictionaire_auxiliary_variable={}
res=1

def variable_name(n,prefix):  # np est len de p en binaire, nq est len de q en binaire
    var_name_list = []
    for i in range(n):
        var_name_list.append(prefix+str(i+1))
    return var_name_list

def create_variables(var_name):
    if isinstance(var_name, str) :
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol

def create_varibales_from_list(list):
    for element in list:
        create_variables(element)
    return

def reduct_variable_power_term(term):
    new_term = 1
    for variable,power in term.as_powers_dict().items():
        new_term *= variable
    return new_term

def reduct_variable_power_function(expr):
    expanded_expr = sp.expand(expr)
    expanded_reducted = 0
    for term in expanded_expr.args:
        new_term = reduct_variable_power_term(term)
        expanded_reducted += new_term
    return expanded_reducted

def sort_key(variable):
    name = str(variable)
    # Split le nom du variable à deux parties: letter & number
    letter = name[0]
    number = int(name[1:])
    return (letter, number)

def rest_term(coef , rest_variable_liste):
    new_term = coef
    while(len(rest_variable_liste)!=0):
        new_term = new_term * rest_variable_liste[0]
        rest_variable_liste.pop()
    return new_term

def reduct_demension_term(coef,variables_liste):
    global res

    # sélection les 3 premières variables
    [x_1, x_2, x_3] = variables_liste[:3]

    # garder le reste des variables
    rest_variables_liste= variables_liste[3:]
    rest_expr = rest_term(coef , rest_variables_liste)

    # commence la réduction dimension
    tup=(x_1,x_2)
    #check dict s'il existe déjà une variable auxiliary qui = les deux meme variable
    key = next((k for k, v in dictionaire_auxiliary_variable.items() if v == tup), None)
    if key==None:#sinon, il faut creer une variable auxiliary
        var_name = "x" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]=(x_1,x_2)
        res += 1
    else:
        x_4 = key

    expr_reduct_formula = x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)

    expr_reduct = rest_expr * expr_reduct_formula
    return sp.expand(expr_reduct)

def reduct_demension_expression(expr):
    new_expr = 0
    need_reduction = False

    for term in expr.args:
        # get coefficant(la constante) et variables(dans une liste)
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]

        # pour les termes qui ont plus de 2 variables, il faut appliquer la formule de réduction
        if len(variables_liste) > 2 :
            if len(variables_liste) > 3 :
                need_reduction = True
            new_term = reduct_demension_term(coef, variables_liste)
            new_expr +=  new_term
        else:
            # sinon, on ne fait rien et les met directement dans la nouvelle expression
            new_expr += term

    if need_reduction: # récursive
        return reduct_demension_expression(new_expr)
    else:
        return new_expr

def reduct_expression(expr_init):
    expr = sp.expand(expr_init)
    expr_reduct_power = reduct_variable_power_function(expr)
    reduct_dimension_expr = reduct_demension_expression(expr_reduct_power)
    return reduct_dimension_expr

def transfers_qubo2Ising(reduct_demension_expr):
    subs_relations = {(variable, ( variable + 1 ) /2) for variable in variables}

    # Après la substitution, variables dans l'expression sont des variables d'Ising dans {-1,+1}
    new_expr = reduct_demension_expr.subs(subs_relations)
    new_expr = sp.expand(new_expr)
    return new_expr

# action

Exemple: N= 143 = 13 * 11

In [2]:
def set_variable(l,prefix):
    # length de p et q sont suposés logN/2
    #length_N = math.ceil(math.log2(N))
    #l1 = l2 = math.ceil(length_N/2)

    expr = 1
    var_binary = [1]
    for i in range(1,l-1):
        var_name = (prefix+ str(i))
        var = create_variables(var_name)
        expr = 2**i * var + expr
        var_binary.append(var)
    expr = expr + 2 ** (l-1)
    var_binary.append(1)
    var_binary = var_binary[::-1]
    return (expr,var_binary)

def get_N_binary(N):
    binary_str = np.binary_repr(N)
    binary_list = [int(bit) for bit in binary_str]
    return binary_list

In [3]:
# exemple : 35 = 5 * 7
(p_expr,p_binary) = set_variable(3,"p")
print(f"p_expr = {p_expr}")
print(f"p_binary = {p_binary}")

(q_expr,q_binary) = set_variable(3,"q")
print(f"q_expr = {q_expr}")
print(f"q_binary = {q_binary}")

N_binary = get_N_binary(35)
print(f"N_binary = {N_binary}")

N_binary_transposed = N_binary[::-1]
print(f"N_binary_transposed = {N_binary_transposed}")

p_expr = 2*p1 + 5
p_binary = [1, p1, 1]
q_expr = 2*q1 + 5
q_binary = [1, q1, 1]
N_binary = [1, 0, 0, 0, 1, 1]
N_binary_transposed = [1, 1, 0, 0, 0, 1]


## table de mulp

ligne par ligne

In [4]:
def table_multiplication_ligne(coef,p_binary):
    lst = [coef * element for element in p_binary]
    return lst

def table_multiplication(p_binary,q_binary):
    table = []
    lst = q_binary.copy()
    res = 0
    while not lst == []:
        element = lst.pop()
        ele = table_multiplication_ligne(element,p_binary)
        for i in range(res):
            ele.append(0)
        table.append(ele)
        res+=1
    #table.append(N_binary)
    return table


In [5]:
# N=35
table = table_multiplication(p_binary,q_binary)
table

[[1, p1, 1], [q1, p1*q1, q1, 0], [1, p1, 1, 0, 0]]

In [6]:
def adjust_table(table):
    max_cols = max(len(row) for row in table)
    complet_table = [[0]*(max_cols - len(row)) + row for row in table]
    return complet_table

In [7]:
# N=35
table_complet = adjust_table(table)
table_complet

[[0, 0, 1, p1, 1], [0, q1, p1*q1, q1, 0], [1, p1, 1, 0, 0]]

## Transferme à une table de mulptication colonne

In [8]:
def define_table_transposed(table_complet):
    table_transposed = [[row[i] for row in table_complet] for i in range(len(table_complet[0]))]
    table_transposed = table_transposed[::-1]
    return table_transposed


In [9]:
# N=35
table_transposed = define_table_transposed(table_complet)
table_transposed

[[1, 0, 0], [p1, q1, 0], [1, p1*q1, 1], [0, q1, p1], [0, 0, 1]]

## Explore la table par bloc

Suivant le nombre des variables(de p et q) par bloc et les bit de N qui se situ sur les meme position, on calcule le nombre de variables "carry" qu'on doit créer.

Sommes max : on suppose que tout les variables ont valeur 1. suivant les positions des variables, elle portent des poid différents.


###Si la dernière bit de cette sommes ait la même bit que la bit de N du colonne, la sommes max = cette sommes. Sinon, la sommes max = cette sommes -1. le nombre de carry variables est donc le nombre de bit de sommes max -1.

In [10]:
def num_carry(table,N_binary_transposed,first_num_col, num_col_bloc):
    expr = sum(table[first_num_col])
    for i in range(num_col_bloc):
        num_col = first_num_col + i
        expr += sum(table[num_col]) * 2 ** i

    if isinstance(expr, (int)):
        max = expr
    else:
        # suppose all the terms have value 1 so that the number of the terms is the max value of the sum of the column
        variables_res = expr.free_symbols
        substitution_dict = {var: 1 for var in variables_res}
        max = expr.subs(substitution_dict)

    # ici on peux améliorer par rapport au papier: c'est de trouver un max du bloc qui a les meme dernière bits que les meme bloc de bit de N
    binary_str = np.binary_repr(max)
    last_bit = binary_str[-1]
    if int(last_bit) != N_binary_transposed[first_num_col+num_col_bloc]:
        max -= 1

    num = len(np.binary_repr(max)) - num_col_bloc
    return num

def define_carry_var(num_carry):
    global c_res
    carry_var = []
    for i in range(num_carry):
        var = create_variables("c" + str(c_res))
        carry_var.append(var)
        c_res+=1
    return carry_var

## Table de mulplication par colonne avec les variables de carry

faut par bloc

In [15]:
def define_carry_table(table_transposed,num_per_bloc,N_binary):
    global c_res
    c_res = 1

    #table_res = table_transposed.copy()
    table_res = copy.deepcopy(table_transposed)
    columns = len(table_transposed)

    N_binary_transposed = N_binary[::-1]
    #obj_function = 0

    #begain from the 2nd col
    first_col_num = 1

    while first_col_num < columns:
        num = num_carry(table_res,N_binary_transposed,first_col_num,num_per_bloc)
        if num > 0:
            lst_carry_var = define_carry_var(num) #a list contains the created carry variables.
            for j in range(num):
                carry_col = first_col_num + num_per_bloc + j
                if carry_col <= len(table_res)- 1:
                    table_res[carry_col].append(lst_carry_var[j])
                elif len(table_res) < len(N_binary_transposed):
                    table_res.append([lst_carry_var[j]])# new column

        first_col_num += num_per_bloc
    return table_res

table_carry = define_carry_table(table_transposed,2,N_binary)
table_carry

[[1, 0, 0], [p1, q1, 0], [1, p1*q1, 1], [0, q1, p1, c1], [0, 0, 1, c2], [c3]]

In [16]:
#N=35
table_carry = define_carry_table(table_transposed,2,N_binary)
table_carry

[[1, 0, 0], [p1, q1, 0], [1, p1*q1, 1], [0, q1, p1, c1], [0, 0, 1, c2], [c3]]

## Define function

On peut définir la fonction objective.

Pour chque bloc, on peut définir une équation : la sommes des variables de p et q = la sommes des variables de carry et les bit de N (suivant la position de bit, les variables ont des poids différent)

In [17]:
def define_function(table_carry,num_col_per_bloc,N_binary_transposed):
    columns = len(table_carry)
    first_col_num = 1
    obj_function = 0

# per bloc
    while first_col_num < columns:
        lhs = 0
        rhs = 0

        for i in range(num_col_per_bloc):
            col_num = first_col_num + i
            if col_num < len(table_carry):
                lhs += sum(table_carry[col_num]) * 2 ** i
                rhs += N_binary_transposed[col_num] * 2 ** i                # N_binary

                if col_num+num_col_per_bloc < len(table_carry):
                    carry_level = table_carry[col_num + num_col_per_bloc]   # 2 is the num_col_per_bloc
                    carry_list=[]
                    for ele in carry_level:
                        if isinstance(ele,sp.Symbol):
                            if ele.name.startswith("c"):
                                carry_list.append(ele)
                    for element in carry_list:
                        rhs += element * 2**(2+i)
                else: # it's the last column of the last bloc
                    # comple the last bit of the N_binary
                    if col_num + 1 < len(N_binary_transposed):
                        rhs += N_binary_transposed[col_num + 1] * 2 ** (i + 1)

        term = (lhs - rhs)**2
        obj_function+=term

        first_col_num += num_col_per_bloc
    return obj_function

In [18]:
expr = define_function(table_carry,2,N_binary_transposed)
expr

(c3 - 1)**2 + (-4*c1 - 8*c2 + 2*p1*q1 + p1 + q1 + 3)**2 + (c1 + 2*c2 - 4*c3 + p1 + q1 - 2)**2

 # forme quadratique

réduction de power et >2terme

In [19]:
expr = sp.expand(expr)
expr

17*c1**2 + 68*c1*c2 - 8*c1*c3 - 16*c1*p1*q1 - 6*c1*p1 - 6*c1*q1 - 28*c1 + 68*c2**2 - 16*c2*c3 - 32*c2*p1*q1 - 12*c2*p1 - 12*c2*q1 - 56*c2 + 17*c3**2 - 8*c3*p1 - 8*c3*q1 + 14*c3 + 4*p1**2*q1**2 + 4*p1**2*q1 + 2*p1**2 + 4*p1*q1**2 + 16*p1*q1 + 2*p1 + 2*q1**2 + 2*q1 + 14

In [20]:
expr_reduct_power = reduct_variable_power_function(expr)
expr_reduct_power

68*c1*c2 - 8*c1*c3 - 16*c1*p1*q1 - 6*c1*p1 - 6*c1*q1 - 11*c1 - 16*c2*c3 - 32*c2*p1*q1 - 12*c2*p1 - 12*c2*q1 + 12*c2 - 8*c3*p1 - 8*c3*q1 + 31*c3 + 28*p1*q1 + 4*p1 + 4*q1 + 14

In [21]:
reduct_dimension_expr = reduct_demension_expression(expr_reduct_power)
reduct_dimension_expr

68*c1*c2 - 8*c1*c3 - 38*c1*p1 - 6*c1*q1 + 64*c1*x2 - 11*c1 - 16*c2*c3 - 76*c2*p1 - 12*c2*q1 + 128*c2*x1 + 12*c2 - 8*c3*p1 - 8*c3*q1 + 31*c3 + 28*p1*q1 + 128*p1*x1 + 64*p1*x2 + 4*p1 - 32*q1*x1 - 16*q1*x2 + 4*q1 - 192*x1 - 96*x2 + 14

# Ising modèle

In [22]:
expr_ising = transfers_qubo2Ising(reduct_dimension_expr)
expr_ising

17*c1*c2 - 2*c1*c3 - 19*c1*p1/2 - 3*c1*q1/2 + 16*c1*x2 + 29*c1/2 - 4*c2*c3 - 19*c2*p1 - 3*c2*q1 + 32*c2*x1 + 29*c2 - 2*c3*p1 - 2*c3*q1 + 11*c3/2 + 7*p1*q1 + 32*p1*x1 + 16*p1*x2 + 53*p1/2 - 8*q1*x1 - 4*q1*x2 - 19*q1/2 - 40*x1 - 20*x2 - 45

# solver

In [23]:
import dimod
from dwave.preprocessing.composites import SpinReversalTransformComposite

# Compose the sampler
base_sampler = dimod.ExactSolver()
composed_sampler = SpinReversalTransformComposite(base_sampler)

def get_solver_parameter(expr_ising):
    linear = {}
    quadratic ={}
    offset = 0

    for term in expr_ising.args:
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]
        if len(term.free_symbols) ==0:
            offset = coef
        if len(term.free_symbols) ==1:
            linear[variables_liste[0].name] = coef
        elif len(term.free_symbols) ==2:
            quadratic[(variables_liste[0].name,variables_liste[1].name)] = coef

    return (linear,quadratic,offset)

def get_solution(expr_ising):
    (linear,quadratic,offset) = get_solver_parameter(expr_ising)
    response = composed_sampler.sample_ising(linear, quadratic)  #response.data() return (solution, energy, num_occurrences)
    solution = next(response.data())[0]
    return solution

In [24]:
solution = get_solution(expr_ising)
solution

{'c2': np.int8(-1),
 'p1': np.int8(-1),
 'q1': np.int8(1),
 'x1': np.int8(1),
 'c3': np.int8(-1),
 'x2': np.int8(1),
 'c1': np.int8(-1)}

In [25]:
def calcule_p_q(solution,p,q):
    dic_varialbe_value = {}
    for var in variables:
        valeur_ising = solution.get(var.name, np.int8(-1))

        if(valeur_ising == np.int8(1)):
            dic_varialbe_value[var] = int(1)
        else:
            dic_varialbe_value[var] = int(0)

    p_value = p.subs(dic_varialbe_value)
    q_value = q.subs(dic_varialbe_value)

    return (p_value.evalf(),q_value.evalf())

In [26]:
calcule_p_q(solution,p_expr,q_expr)

(5.00000000000000, 7.00000000000000)

# Sum up function

In [27]:
def factorization(N,p_len,q_len,num_col_per_bloc):
    global p_expr,q_expr,N_binary,N_binary_transposed,variables, dictionaire_auxiliary_variable, res

    (p_expr,p_binary) = set_variable(p_len,"p")
    (q_expr,q_binary) = set_variable(q_len,"q")
    N_binary = get_N_binary(N)
    N_binary_transposed = N_binary[::-1]

    table = table_multiplication(p_binary,q_binary)
    table_complet = adjust_table(table)
    table_transposed = define_table_transposed(table_complet)
    table_carry = define_carry_table(table_transposed,num_col_per_bloc,N_binary)
    print(table_carry)

    expr_init = define_function(table_carry,num_col_per_bloc,N_binary_transposed)
    reduct_expr = reduct_expression(expr_init)

    expr_ising = transfers_qubo2Ising(reduct_expr)

    solution = get_solution(expr_ising)
    (p,q) = calcule_p_q(solution,p_expr,q_expr)

    return p,q

factorization(35,3,3,2)

[[1, 0, 0], [p1, q1, 0], [1, p1*q1, 1], [0, q1, p1, c1], [0, 0, 1, c2], [c3]]


(5.00000000000000, 7.00000000000000)

In [28]:
factorization(143,4,4,2)

[[1, 0, 0, 0], [p1, q1, 0, 0], [p2, p1*q1, q2, 0], [1, p2*q1, p1*q2, 1, c1], [0, q1, p2*q2, p1, c2], [0, 0, q2, p2, c3], [0, 0, 0, 1, c4], [c5, c6]]


(9.00000000000000, 15.0000000000000)